In [1]:
import sys
from os import path, mkdir, listdir, chdir
import pickle
import cv2 as cv
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from skimage import io
import pandas as pd
import numpy as np

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)
    
from utiles import lectura_img



In [2]:
#Recibe lista de keypoints de una imagen, crea los objetos KP de openCV
def genKeyPoints( kp_img ):
    keypoints = list()
    for kp in kp_img:
        keypoints.append(cv.KeyPoint(kp[0], kp[1], kp[2]))
    return keypoints

## Visualización de descriptores
#### Rutas de ejecución



In [3]:
@interact_manual
def load_images(pickle_path='', image_dir='', 
                image_dir_out=''):
    
    
    pickle_file = open(path.abspath(pickle_path), 'rb')  
    args = pickle.load(pickle_file)
    desc_imgs = pickle.load(pickle_file)
    pickle_file.close()

    indexes = range(len(desc_imgs))
    for i in indexes:
        imagen = path.abspath(image_dir + desc_imgs[i]['name_img'])
        img = cv.imread(imagen)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        keypoints = genKeyPoints(desc_imgs[i]['keypoints'])
        img=cv.drawKeypoints(gray,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv.imwrite(image_dir_out+desc_imgs[i]['name_img'],img)
    
    global dir_out
    dir_out=image_dir_out
    print('Listo!' )

interactive(children=(Text(value='', description='pickle_path'), Text(value='', description='image_dir'), Text…

In [5]:
@interact
def show_images(img=listdir(dir_out)):
    chdir(dir_out)
    image=io.imread(path.abspath(img))
    plt.imshow(image)

NameError: name 'dir_out' is not defined

## Visualización tópicos
#### Rutas de ejecución

1. /media/working/minia/minhash/index_inv_full_dataset_v2_SIFT.r3l1000.models
2. /media/working/minia/descriptores/labels_full_dataset_v2_SIFT.pickle
3. /media/working/minia/descriptores/full_dataset_v2_SIFT.pickle
4. /media/working/minia/images/images_training_rev1/
5. /media/working/minia/images/pruebas


In [4]:
@interact_manual
def load_topicos(topics_path='',centXimg='', descr_kp='', image_dir='', 
                img_dir_out=''):
    global topics
    global len_topics
    global images_descr
    global desc_kp
    global etiquetas
    global dir_out
    global dir_img
    global topics_lenght
    
    topics = open(topics_path, 'r')
    topics= topics.readlines()
    topics_lenght=[]
    for line in topics:
        centroides = line.strip().split()[1:]
        centroides = [ int(cent.split(':')[0]) for cent in centroides]
        topics_lenght.append(len(centroides))
        
    len_topics=len(topics)
    
    pickle_file = open(centXimg,'rb')
    images_descr = pickle.load(pickle_file)
    pickle_file.close()
    
    pickle_file = open(path.abspath(descr_kp), 'rb')  
    args = pickle.load(pickle_file)
    desc_kp = pickle.load(pickle_file)
    pickle_file.close()
    
    dir_out=img_dir_out
    dir_img=image_dir

    print('#Topics: '+ str(len_topics))

interactive(children=(Text(value='', description='topics_path'), Text(value='', description='centXimg'), Text(…

In [21]:
def indicesTopicos (centroides, images_descr,threshold=0.7):
    tam_cents = len(centroides)
    centroides_= set(centroides) 
    img_index = 0
    images = list()
    for descrp in images_descr[0]:
        descp_ = dict([(ic,i) for i,ic in enumerate(descrp)])
        descp = set(descrp)
        tam_descp = len(descrp)
        inter=centroides_.intersection(descp)
        overlap=len(inter)/min(len(centroides),len(centroides))
        if overlap >= threshold:
            images.append((img_index,[ descp_[ic] for ic in inter],overlap))
        #Overlaping
        #max_ = round( min(tam_descp, tam_cents)*.30 ) #Requiere un 30%
        #posc_index = 0
        #posc = list()
        #for cent in descrp:
        #    if cent in centroides:
        #        posc.append(posc_index)
        #    posc_index += 1
        #if len(posc) > max_ : #No puede ser >= porque max_ puede ser cero
        #    images.append((img_index ,posc)) #Añade una tupla, del indice de la imagen y los indices de los KP
        img_index += 1
    return images

In [38]:
@interact_manual
def choose_topicos(Topics=[(f"{i} [size {z}]",i) for i,z in zip(range(len_topics),topics_lenght)], num_imgs= (0, int(len(images_descr)/1), 5), threshold = (0.0, 1.0, 0.01)):
    global lista_imgs
    global num_topic
    global name_images
    
    num_topic= Topics
    centroides = topics[Topics].split()[1:]
    centroides = [ int(cent.split(':')[0]) for cent in centroides]
    lista_imgs = indicesTopicos (centroides, images_descr[0:num_imgs], threshold=threshold)
    
    imgs_index=[imgs[0] for imgs in lista_imgs]
    aux=0
    name_images=[]
    for ii,i in enumerate(imgs_index):
        imagen = path.abspath(dir_img + desc_kp[i]['name_img'])
        name_images.append((desc_kp[i]['name_img'],aux,i,lista_imgs[ii][2]))
        aux+=1
        
    print('#Images: '+str(len(lista_imgs)))
    print('#Len topic: '+str(len(centroides)))

interactive(children=(Dropdown(description='Topics', options=(('0 [size 179]', 0), ('1 [size 38]', 1), ('2 [si…

In [42]:
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['figure.dpi'] = 160

@interact
def show_images_per_topic(img=[(aux,(img,aux,i,o)) for img,aux,i,o in name_images]):
    img,aux,i,o=img
    if aux-5 < 0:
        aux=5
    if aux+5 > len(name_images):
        aux=len(name_images)-5
    min_=max(0,aux-5)
    max_=min(len(name_images),aux+5)
    fig, axs = plt.subplots(2,5)
    for ii in range(min_,max_):
        img,aux,i,overlap=name_images[ii]
        imagen = path.abspath(dir_img + img)
        img = cv.imread(imagen)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        key_points= [desc_kp[i]['keypoints'][kp] for kp in lista_imgs[aux][1]]
        keypoints = genKeyPoints(key_points)
        img=cv.drawKeypoints(gray,keypoints,img,color=(255,0,0),flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        x=ii-min_
        y=x%5
        x=int(x/5)
        axs[x,y].set_yticklabels([])
        axs[x,y].set_xticklabels([])
        axs[x,y].set_xlabel(f'{ii}, {overlap:0.3}')
        axs[x,y].imshow(img)

interactive(children=(Dropdown(description='img', options=((0, ('477308.jpg', 0, 195, 0.38461538461538464)), (…

In [ ]:
@interact
def show_images_per_topic2(img=listdir(dir_out)):
    chdir(dir_out)
    image=io.imread(path.abspath(img))
    plt.imshow(image)